In [30]:
import rsa
import requests as r

bob has a set of keys

In [31]:
(bob_pub, bob_priv) = rsa.newkeys(512)

In [32]:
print(bob_pub)

PublicKey(8002785484210848775085432105827485824029293523180931063636319441126549998344542187099056227525916646580600754502232945474789420466117219397245460223207251, 65537)


In [33]:
print(bob_priv)

PrivateKey(8002785484210848775085432105827485824029293523180931063636319441126549998344542187099056227525916646580600754502232945474789420466117219397245460223207251, 65537, 3811571266676189536681823668329327593744760486939737586711995895987367921176459111551947665568421175540939264365167829024860910977450717442209941411684161, 6349232363522354524099252044721057833187756153215583025226083255465935173960068419, 1260433549445834891427476010034281879373888604417672598229698415618634929)


Alice has a set of keys as well!

In [34]:
(alice_pub, alice_priv) = rsa.newkeys(512)

In [35]:
print(alice_pub)

PublicKey(7808271758120995803548296984620924590378801245188678512209496135637766710449861205000670879245432343894686471716686278098721307347879373171755780550823477, 65537)


In [36]:
print(alice_priv)

PrivateKey(7808271758120995803548296984620924590378801245188678512209496135637766710449861205000670879245432343894686471716686278098721307347879373171755780550823477, 65537, 672919402625499859597491209074858203556151020535356458747871189924502286344671237036691847831347718653204800312637515787487250985779821933768993585597953, 7161844408288301626692324481822135781494802710308749756315123873287673815584448937, 1090259898565318301630786703308051260395397902333206599375332224771733421)


Alice can send a secure message to bob by encrypting her message using **bob's** public key - only bob will be able to decrypt it!

In [58]:
quote = "no amount of violence will ever solve a math problem.".encode("utf-8")
encrypted_message = rsa.encrypt(quote,bob_pub) #think of this as "locking" the message with the public key
encrypted_message

b"3i?\x9e+\xa7\x80\xa91\x8e\x82\xdb\xd7X\xc0\x00\xff\xec4\xfb\xd3\xa2\x01\x86\xed\x80\xf6\xef\x0f\xf3\xd1\xad\x93\xab\xc5`\x8eh\xef\xfa<\xa8\x1f\x93\xd4\xdb\x90\xff'P\xf2\x07\xf5\x95\xd9\xec;\xfbv\x97\xf1{)N"

In [59]:
bob_pub_pem = bob_pub._save_pkcs1_pem()
bob_pub_pem

b'-----BEGIN RSA PUBLIC KEY-----\nMEgCQQCYzMz1K6HtGAykYwtTJW1AbAShjePZ9vIMXOMPsxSvspIKLqaSD9mn+9uW\n39BwahscAvSz9pA2//WIKcoU6adTAgMBAAE=\n-----END RSA PUBLIC KEY-----\n'

In [60]:
alice_pub_pem = alice_pub._save_pkcs1_pem()
alice_pub_pem

b'-----BEGIN RSA PUBLIC KEY-----\nMEgCQQCVFgmOUyKzNPlUov4YBAMO5lmxph32Ud89cRbnvtSuXpcv6It4g3gQDXTB\nnKN0DZt+C5OWo87rKqfhITju7g41AgMBAAE=\n-----END RSA PUBLIC KEY-----\n'

We can send this encrypted message over the blockchain less fear of identification or decryption by a third party. Only the two parties involved know who the sender and recipient are, and only bob has the private key to decrypt the message. In this case, the public key is being used both as an address and an encryption tool!

In [62]:
payload = {
    "from": alice_pub_pem,
    "to": bob_pub_pem,
    "msg":encrypted_message
}

res = r.post("http://localhost:5000/new",data=payload)

In [63]:
from pprint import pprint
res.json()["received"]["from"]

'-----BEGIN RSA PUBLIC KEY-----\nMEgCQQCVFgmOUyKzNPlUov4YBAMO5lmxph32Ud89cRbnvtSuXpcv6It4g3gQDXTB\nnKN0DZt+C5OWo87rKqfhITju7g41AgMBAAE=\n-----END RSA PUBLIC KEY-----\n'

In [64]:
rsa.decrypt(encrypted_message,bob_priv)

b'no amount of violence will ever solve a math problem.'

When you need to take someone's public key and load it into RSA so you can use it for encryption:

In [65]:
rsa.PublicKey.load_pkcs1(bytes("-----BEGIN RSA PUBLIC KEY-----\nMEgCQQCVFgmOUyKzNPlUov4YBAMO5lmxph32Ud89cRbnvtSuXpcv6It4g3gQDXTB\nnKN0DZt+C5OWo87rKqfhITju7g41AgMBAAE=\n-----END RSA PUBLIC KEY-----\n", "utf-8"))

PublicKey(7808271758120995803548296984620924590378801245188678512209496135637766710449861205000670879245432343894686471716686278098721307347879373171755780550823477, 65537)